In [21]:
import pandas as pd

file_path = '../09. Representasi Pengetahuan (Knowledge Representation)/sorted_topic_cluster_counts.csv'

data = pd.read_csv(file_path)

data

,conversation_id_str,created_at,favorite_count,full_text,id_str,lang,location,probability,quote_count,reply_count,retweet_count,topic,tweet_url,user_id_str,username,in_reply_to_screen_name,image_url,context,emotion
0,1800679330726715550,Tue Jun 11 23:59:42 +0000 2024,1,TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY,1800679330726715550,in,Indonesia,0,0,0,0,1,https://x.com/tigapuluhtigax/status/1800679330...,1188470356668895234,tigapuluhtigax,NaN,NaN,Perjalanan timnas Indonesia di babak kualifika...,Joy
1,1800679305925800015,Tue Jun 11 23:59:36 +0000 2024,1,ngerasa ga sih sekarang timnas kalo main pakek...,1800679305925800015,in,"Jombang, Indonesia",0,0,0,0,3,https://x.com/sotongism/status/180067930592580...,961056397063409664,sotongism,NaN,NaN,Anang sering disebut dalam konteks menyanyikan...,Fear
2,1800679288326476196,Tue Jun 11 23:59:31 +0000 2024,0,2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...,1800679288326476196,in,NaN,0,0,0,0,3,https://x.com/peanutkacang123/status/180067928...,1464596330421100548,peanutkacang123,NaN,NaN,Anang sering disebut dalam konteks menyanyikan...,Joy
3,1800589420766355730,Tue Jun 11 23:59:28 +0000 2024,0,@womensfootie_id Tinggal tmbah pemain diaspora...,1800679273185047018,in,NaN,0,0,0,0,1,https://x.com/PKananku1927/status/180067927318...,1772880651248939008,PKananku1927,womensfootie_id,NaN,Perjalanan timnas Indonesia di babak kualifika...,Neutral
4,1800679270857286043,Tue Jun 11 23:59:27 +0000 2024,0,Timnas Indonesia Vs Timnas Filipina 2-0 Indone...,1800679270857286043,in,NaN,0,0,0,0,2,https://x.com/kangsil2012/status/1800679270857...,546890263,kangsil2012,NaN,NaN,Banyak pembicaraan tentang keberhasilan timnas...,Joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,1800537380392419660,Tue Jun 11 15:56:14 +0000 2024,0,@cingreborn aaak seneng banget guee #timnasday,1800557666047254585,in,NaN,0,0,0,0,0,https://x.com/kidleaderr/status/18005576660472...,1294225010924466177,kidleaderr,cingreborn,NaN,Dukungan dari suporter untuk timnas Indonesia ...,Joy
1876,1800557640936038793,Tue Jun 11 15:56:08 +0000 2024,0,PREDIKSI TOGEL OREGON 04 12 JUNI 2024 ANGKA MA...,1800557640936038793,in,NaN,0,0,0,0,0,https://x.com/yupitoto88/status/18005576409360...,1746107999226085376,yupitoto88,NaN,https://pbs.twimg.com/media/GPzdn55aMAA1CUe.png,Dukungan dari suporter untuk timnas Indonesia ...,Neutral
1877,1800557615824744466,Tue Jun 11 15:56:03 +0000 2024,0,Timnas menang tapi aku tetep kalah karo dee si...,1800557615824744466,in,NaN,0,0,0,0,0,https://x.com/Lalalan111111/status/18005576158...,1552276761194274816,Lalalan111111,NaN,NaN,Dukungan dari suporter untuk timnas Indonesia ...,Sad
1878,1800557601232691595,Tue Jun 11 15:55:59 +0000 2024,0,buat mas mas belakang gue minta aidrop gol ked...,1800557601232691595,in,NaN,0,0,1,0,0,https://x.com/celestiaalsky/status/18005576012...,1761777254135394304,celestiaalsky,NaN,NaN,Dukungan dari suporter untuk timnas Indonesia ...,Sad


In [22]:
# Import libraries yang diperlukan
import string
import pandas as pd
import numpy as np
import re
import nltk
import pickle
from nltk.tokenize import word_tokenize
from mpstemmer import MPStemmer
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Unduh stopwords dari NLTK
nltk.download('stopwords')
stemmer = MPStemmer()

# Memuat model analisis emosi yang sudah dilatih
model = load_model('../07. Pemodelan Data (Data Mining)/model/model-cnn.h5')

class Emotion:
    @staticmethod
    def classify_emotion(data):

        def lower_case(text):
            return text.lower()

        def remove_tweet_special(text):
            text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', "")
            text = text.encode('ascii', 'replace').decode('ascii')
            text = ' '.join(re.sub(r"([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
            return text.replace("http://", " ").replace("https://", " ")

        def remove_number(text):
            return re.sub(r"\d+", "", text)

        def remove_punctuation(text):
            return text.translate(str.maketrans("", "", string.punctuation))

        def remove_whitespace_LT(text):
            return text.strip()

        def remove_whitespace_multiple(text):
            return re.sub(r'\s+', ' ', text)

        def remove_singl_char(text):
            return re.sub(r"\b[a-zA-Z]\b", "", text)

        def remove_repeated_char(text):
            return re.sub(r'(.)\1+', r'\1', text)

        def word_tokenize_wrapper(text):
            return word_tokenize(text)

        normalizad_word = pd.read_csv("./utils/kamus-alay.csv")
        normalizad_word_dict = {}

        for index, row in normalizad_word.iterrows():
            if row[0] not in normalizad_word_dict:
                normalizad_word_dict[row[0]] = row[1]

        def normalized_term(document):
            return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

        def stem_wrapper(term):
            return [stemmer.stem(word) for word in term]

        stop_words = stopwords.words('indonesian')
        stop_words = [word for word in stop_words if word not in ['tidak', 'baik', 'jelek', 'jangan', 'belum', 'bukan', "enggak", "engga", "bener", "benar"]]
        stop_words.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                            'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                            'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                            '&amp', 'yah'])

        txt_stopword = pd.read_csv("utils/stopwords.txt", names=["stopwords"], header=None)
        stop_words.extend(txt_stopword["stopwords"][0].split(' '))
        stop_words = set(stop_words)

        def stopwords_removal(words):
            return [word for word in words if word not in stop_words]
        
        def replace_nan_with_none(data):
            return data.applymap(lambda x: None if pd.isna(x) else x)

        df = pd.DataFrame(data)

        if 'predicted_label' not in df.columns:
            df['predicted_label'] = np.nan
            df['probability_emotion'] = np.nan

        to_process_df = df[df['predicted_label'].isna()]

        if not to_process_df.empty:
            to_process_df['processed_text'] = to_process_df['full_text'].apply(lower_case)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_tweet_special)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_number)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_punctuation)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_whitespace_LT)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_whitespace_multiple)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_singl_char)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_repeated_char)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(word_tokenize_wrapper)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(normalized_term)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(stem_wrapper)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(stopwords_removal)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(' '.join)

            print("Text preprocessing done!")

            with open('../07. Pemodelan Data (Data Mining)/tokenizer-emotion(cnn).pickle', 'rb') as handle:
                tokenizer = pickle.load(handle)

            sequences = tokenizer.texts_to_sequences(to_process_df['processed_text'])
            padded_sequences = pad_sequences(sequences, maxlen=64, truncating='post', padding='post')

            predictions = model.predict(padded_sequences)
            
            emotion_labels = ['Neutral', 'Anger', 'Joy', 'Love', 'Sad', 'Fear']
            predicted_labels = []
            predicted_probabilities = []

            for pred in predictions:
                max_idx = np.argmax(pred)
                predicted_labels.append(emotion_labels[max_idx])
                predicted_probabilities.append({emotion_labels[i]: pred[i] for i in range(len(emotion_labels))})

            print("Prediction done!")

            to_process_df['predicted_label'] = predicted_labels
            to_process_df['probability_emotion'] = [predicted_probabilities[i][predicted_labels[i]] for i in range(len(predicted_labels))]

            df.update(to_process_df)

        df = replace_nan_with_none(df)

        return df
    
    @staticmethod
    def calculate_emotion_percentages(data):
        total = len(data)
        emotion_counts = {'Neutral': 0, 'Anger': 1, 'Joy': 2, 'Love': 3, 'Sad': 4, 'Fear': 5}

        for item in data:
            emotion_counts[item['predicted_label']] += 1
        
        percentages = {emotion: (count / total) * 100 for emotion, count in emotion_counts.items()}
        return percentages


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TEMP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# Mengklasifikasikan emosi
hasil = Emotion.classify_emotion(data)

# Menyimpan hasil hanya full_text, prediksi dan probabilitas 
hasil = hasil[['username','full_text','topic', 'predicted_label', 'probability_emotion']]
hasil.to_csv('hasil_prediksi(cnn).csv', index=False)

C:\Users\TEMP\AppData\Local\Temp\ipykernel_15424\3301233485.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalizad_word_dict:
C:\Users\TEMP\AppData\Local\Temp\ipykernel_15424\3301233485.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalizad_word_dict[row[0]] = row[1]


Text preprocessing done!
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Prediction done!


C:\Users\TEMP\AppData\Local\Temp\ipykernel_15424\3301233485.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Love' 'Anger' 'Joy' ... 'Anger' 'Love' 'Neutral']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.update(to_process_df)
C:\Users\TEMP\AppData\Local\Temp\ipykernel_15424\3301233485.py:83: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return data.applymap(lambda x: None if pd.isna(x) else x)
